<a href="https://colab.research.google.com/github/Sanya003/Small-Medical-LM/blob/main/Small_Medical_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [ ]:
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [ ]:
updated_data = [{'Name': item['Name'], 'Symptoms': item['Symptoms']} for item in data_sample['train']]

In [ ]:
df = pd.DataFrame(updated_data)

In [ ]:
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [ ]:
df['Symptoms'] = df['Symptoms'].apply(lambda x: ', '.join(x.split(', ')))

In [ ]:
df['Symptoms'][0]

'Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness'

In [ ]:
from  transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('distilbert/distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilbert/distilgpt2').to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [ ]:
BATCH_SIZE = 8

In [ ]:
class LanguageDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.labels = df.columns
        self.data = df.to_dict(orient='records')
        self.tokenizer = tokenizer
        x = self.fittest_max_length(df)
        self.max_length = x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]
        y = self.data[idx][self.labels[1]]
        text = f"{x} | {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors="pt", max_length=128, padding="max_length", truncation=True)
        return tokens

    def fittest_max_length(self, df):
        max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
        x = 2
        while x < max_length: x *= 2
        return x

In [ ]:
data_sample = LanguageDataset(df, tokenizer)

In [ ]:
data_sample

In [ ]:
train_size = int(0.8 * len(data_sample))
val_size = len(data_sample) - train_size

train_data, val_data = random_split(data_sample, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

In [ ]:
num_epochs = 4

In [ ]:
batch_size = BATCH_SIZE
model_name = "distilbert/distilgpt2"
gpu = 0

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu', 'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [ ]:
# Training
for epoch in range(num_epochs):
  start_time = time.time()
  model.train()
  epoch_training_loss = 0
  train_iterator = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")

  for batch in train_iterator:
    optimizer.zero_grad()
    inputs = batch['input_ids'].squeeze(1).to(device)
    targets = inputs.clone()
    outputs = model(input_ids = inputs, labels = targets)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    train_iterator.set_postfix({'Training loss': loss.item()})
    epoch_training_loss += loss.item()
  avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

# Validation
  model.eval()
  epoch_validation_loss = 0
  total_loss = 0
  val_iterator = tqdm(val_loader, desc=f"Validating epoch {epoch+1}/{num_epochs}")
  with torch.no_grad():
    for batch in val_iterator:
      inputs = batch['input_ids'].squeeze(1).to(device)
      targets = inputs.clone()
      outputs = model(input_ids = inputs, labels = targets)
      loss = outputs.loss
      total_loss += loss
      val_iterator.set_postfix({'Validation loss': loss.item()})
      epoch_validation_loss += loss.item()
    avg_epoch_validation_loss = epoch_validation_loss / len(val_iterator)

    end_time = time.time()
    epoch_duration_sec = end_time - start_time

    new_row = {
        'transformer': model_name,
        'batch_size': batch_size,
        'gpu': gpu,
        'epoch': epoch+1,
        'training_loss': avg_epoch_training_loss,
        'validation_loss': avg_epoch_validation_loss,
        'epoch_duration_sec': epoch_duration_sec
    }

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}")


Epoch 1/4 Batch Size: 8, Transformer: distilbert/distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.97it/s, Training loss=0.608]
Validating epoch 1/4: 100%|██████████| 10/10 [00:00<00:00, 17.71it/s, Validation loss=0.762]


Epoch: 1, Validation Loss: 0.6759303212165833


Epoch 2/4 Batch Size: 8, Transformer: distilbert/distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.41it/s, Training loss=0.469]
Validating epoch 2/4: 100%|██████████| 10/10 [00:00<00:00, 18.45it/s, Validation loss=0.746]


Epoch: 2, Validation Loss: 0.6430286765098572


Epoch 3/4 Batch Size: 8, Transformer: distilbert/distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.31it/s, Training loss=0.437]
Validating epoch 3/4: 100%|██████████| 10/10 [00:00<00:00, 18.23it/s, Validation loss=0.757]


Epoch: 3, Validation Loss: 0.63362056016922


Epoch 4/4 Batch Size: 8, Transformer: distilbert/distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.28it/s, Training loss=0.346]
Validating epoch 4/4: 100%|██████████| 10/10 [00:00<00:00, 17.95it/s, Validation loss=0.807]

Epoch: 4, Validation Loss: 0.6728989481925964


In [ ]:
input = "Kidney Failure"

In [ ]:
input_ids = tokenizer.encode(input, return_tensors="pt").to(device)

In [ ]:
input_ids

tensor([[48374,  1681, 25743]], device='cuda:0')

In [ ]:
output = model.generate(
    input_ids,
    max_length=64,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
decoded_output

'Kidney Failure | Excessive urination, excessive urine leakage, excessive urine leakage'

In [ ]:
torch.save(model, 'SmallDiseaseLM.pt')